In [1]:
import sys
import time 
import random
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
random.seed(0)
np.random.seed(0)

### spatial queue model
sys.path.insert(0, '../../../')
import model.queue_class_ce170 as sq_loop
import model.spatial_queue_array as sq_vect

In [2]:
#abs_path = os.path.dirname(os.path.abspath(__file__))
#sys.path.insert(0, abs_path)
sys.path.insert(0, "/Users/apple/Documents/GitHub")
from sp import interface

In [3]:
#把bg的终点换成虚拟点，增加node,增加link
ods_df = pd.read_csv('fairfax_ods_background.csv',float_precision="round_trip")
nodes = pd.read_csv('fairfax_nodes.csv',float_precision="round_trip")
links_df = pd.read_csv('fairfax_links.csv',float_precision="round_trip")

In [4]:
### Read network and demand
case='fairfax'
bg_exist = 1 # 1 represents bg exists, 0 represents bg does not exist
nodes_df = pd.read_csv('{}_nodes.csv'.format(case,case))
links_df = pd.read_csv('{}_links.csv'.format(case,case))
od_df = pd.read_csv('{}_ods_day.csv'.format(case,case))

### these are done for the vectorized implementation
if bg_exist == 1:
    background_od = pd.read_csv('{}_ods_background.csv'.format(case,case))
    background_od = background_od.sample(frac=0.1, random_state = 1)
    od_df = pd.concat([od_df, background_od]).sample(frac=1)
od_df.loc[:, 'departure_hour'] = od_df.loc[:, 'departure_hour'].fillna(6)
od_df.loc[:, 'departure_quarter'] = od_df.loc[:, 'departure_quarter'].fillna(0)

od_df['departure_time'] = (od_df['departure_hour'] - 6) * 3600 + od_df['departure_quarter'] * 900

od_df['departure_time'] = np.where(od_df['departure_time']== 900, 10, od_df['departure_time'])

od_df['origin_nid'] = od_df['origin_node_id']
od_df['destin_nid'] = od_df['destin_node_id']
od_df.reset_index(drop = True, inplace = True)
od_df['agent_id'] = np.arange(len(od_df))
nodes_df['node_id'] = nodes_df['nid']
nodes_df['node_osmid'] = None
links_df['link_id'] = links_df['eid']
links_df['start_node_id'] = links_df['nid_s']
links_df['end_node_id'] = links_df['nid_e']
print('# nodes {}, # links {}, # ods {}'.format(nodes_df.shape[0], links_df.shape[0], od_df.shape[0]))

# nodes 224224, # links 549010, # ods 15269


In [5]:
simulation_vect = sq_vect.Simulation()

In [6]:
simulation_vect.initialize_simulation(nodes_df, links_df, od_df)

In [7]:
simulation_vect.agents.agents

,origin_nid,destin_nid,agent_status,agent_id,departure_time,current_link,current_link_enter_time,cl_snid,cl_enid,cl_fft,cl_lanes,next_link,nl_enid,nl_lanes
0,14041,133679,0,0,13500.0,NaN,NaN,NaN,14041,NaN,NaN,NaN,NaN,NaN
1,136192,120150,0,1,13500.0,NaN,NaN,NaN,136192,NaN,NaN,NaN,NaN,NaN
2,132084,224223,0,2,0.0,NaN,NaN,NaN,132084,NaN,NaN,NaN,NaN,NaN
3,138899,224223,0,3,0.0,NaN,NaN,NaN,138899,NaN,NaN,NaN,NaN,NaN
4,212762,122125,0,4,8100.0,NaN,NaN,NaN,212762,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15264,77033,224223,0,15264,0.0,NaN,NaN,NaN,77033,NaN,NaN,NaN,NaN,NaN
15265,111083,111693,0,15265,0.0,NaN,NaN,NaN,111083,NaN,NaN,NaN,NaN,NaN
15266,56637,194511,0,15266,12600.0,NaN,NaN,NaN,56637,NaN,NaN,NaN,NaN,NaN
15267,136212,224223,0,15267,0.0,NaN,NaN,NaN,136212,NaN,NaN,NaN,NaN,NaN


In [8]:
simulation_vect.agents.agents.head(10)

,origin_nid,destin_nid,agent_status,agent_id,departure_time,current_link,current_link_enter_time,cl_snid,cl_enid,cl_fft,cl_lanes,next_link,nl_enid,nl_lanes
0,14041,133679,0,0,13500.0,NaN,NaN,NaN,14041,NaN,NaN,NaN,NaN,NaN
1,136192,120150,0,1,13500.0,NaN,NaN,NaN,136192,NaN,NaN,NaN,NaN,NaN
2,132084,224223,0,2,0.0,NaN,NaN,NaN,132084,NaN,NaN,NaN,NaN,NaN
3,138899,224223,0,3,0.0,NaN,NaN,NaN,138899,NaN,NaN,NaN,NaN,NaN
4,212762,122125,0,4,8100.0,NaN,NaN,NaN,212762,NaN,NaN,NaN,NaN,NaN
5,123678,177167,0,5,6300.0,NaN,NaN,NaN,123678,NaN,NaN,NaN,NaN,NaN
6,170210,109153,0,6,8100.0,NaN,NaN,NaN,170210,NaN,NaN,NaN,NaN,NaN
7,138999,16650,0,7,8100.0,NaN,NaN,NaN,138999,NaN,NaN,NaN,NaN,NaN
8,145378,224223,0,8,0.0,NaN,NaN,NaN,145378,NaN,NaN,NaN,NaN,NaN
9,134096,51875,0,9,6300.0,NaN,NaN,NaN,134096,NaN,NaN,NaN,NaN,NaN


In [9]:
# count the number of evacuees that have successfully reach their destination
def arrival_counts_vect(t, simulation):
    arrival_cnts = np.sum((simulation.agents.agents['agent_status']==-1) & (simulation.agents.agents['destin_nid'] == 224223))
    need_arrival_cnts = np.sum(simulation.agents.agents['destin_nid'] == 224223)
    arrival_cnts_total = np.sum(simulation.agents.agents['agent_status']==-1)
    need_arrival_cnts_total = simulation.agents.agents.shape[0]
    print('At {} seconds, {}-{} evacuees successfully reached the destination'.format(t, arrival_cnts, need_arrival_cnts))
    print('At {} seconds, {}-{} evacuees successfully reached the destination (total)'.format(t, arrival_cnts_total, need_arrival_cnts_total))
    if arrival_cnts == need_arrival_cnts:
        print("all agents arrive at destinations at time {} seconds.".format(t))
        return False
    return True

In [10]:
t_end = 20
for t in range(t_end):
    print(t)
    # run the spatial-queue simulation for one step
    if t == 100:
        simulation_vect.agents.load_agents(network= simulation_vect.network, t=t)
        print(simulation_vect.agents.agents[simulation_vect.agents.agents['departure_time'] == t].head(10))
    simulation_vect.run_one_step(t, reroute_frequency=None)
    if t%100 == 0:
        if not arrival_counts_vect(t, simulation_vect):
            break
        

0
At 0 seconds, 0-4054 evacuees successfully reached the destination
At 0 seconds, 0-15269 evacuees successfully reached the destination (total)
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [11]:
simulation_vect.agents.agents[simulation_vect.agents.agents['departure_time'] == 10]

,origin_nid,destin_nid,agent_status,agent_id,departure_time,current_link,current_link_enter_time,cl_snid,cl_enid,cl_fft,cl_lanes,next_link,nl_enid,nl_lanes,cl_capacity,nl_capacity
56,120904,195901,1,56,10.0,293629,10.0,120904,146178,25.129924,1.0,355447,122040,1.0,0.277778,0.277778
74,130189,143757,1,74,10.0,317185,10.0,130189,165537,188.259959,5.0,401439,126857,4.0,1.388889,1.111111
94,141632,133682,1,94,10.0,301924,16.0,124276,124274,10.054314,1.0,301916,121680,1.0,0.277778,0.277778
171,112845,146185,1,171,10.0,275433,12.0,112845,86600,10.728403,1.0,214863,99049,1.0,0.277778,0.277778
185,211063,109054,1,185,10.0,516762,15.0,211062,211061,10.026929,2.0,516759,211060,2.0,0.555556,0.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15022,127009,5365,1,15022,10.0,309027,10.0,127009,132456,20.122859,1.0,322920,56637,1.0,0.277778,0.277778
15128,113081,117000,1,15128,10.0,275949,10.0,113081,149765,26.350341,5.0,363579,149767,4.0,1.388889,1.111111
15172,141545,107999,1,15172,10.0,344826,10.0,141545,129925,28.419103,1.0,316549,129929,1.0,0.277778,0.277778
15185,58736,11479,1,15185,10.0,144917,13.0,58736,4626,34.177671,1.0,11718,201360,1.0,0.277778,0.277778


In [12]:
route = simulation_vect.agents.get_agent_routes(120904.0, 195901, simulation_vect.network.g)
route

ArgumentError: argument 2: <class 'TypeError'>: Don't know how to convert parameter 2